# Streaming Capabilities Demonstration


This notebook demonstrates how to test the streaming capabilities of the `StreamingClient`. It includes creating Kafka streams, consuming messages, and deleting streams. Sensitive information like usernames and passwords has been replaced with placeholders.


## Setup


Ensure you have the required dependencies installed and a valid API URL. The provided credentials should also be correct for the specific streams you want to test.


In [ ]:
# Required Libraries
import asyncio
import logging
from streaming import StreamingClient
from pointofpresence import APIClient
import time

# Constants
API_URL = "http://155.101.6.190:8002"
USERNAME = "placeholder"  # Replace with your username
PASSWORD = "placeholder"  # Replace with your password

# Stream configurations
streams_to_test = [
    {
        "keywords": "pinguintest,CSV",
        "match_all": True,
        "filter_semantics": [
            "window_filter(5, mean, fixed acidity > 8.7)",
            "residual sugar > 1.5",
            "IF window_filter(9, sum, residual sugar > 20) THEN alert = fixed acidity*100 ELSE fixed acidity = fixed acidity*1000",
            "IF alert IN ['blue'] THEN residual sugar = fixed acidity*100"
        ],
    },
    {
        "keywords": "earthscope_kafka_gnss_observations,kafka",
        "match_all": True,
        "filter_semantics": [],
        "username": "<earthscope_username>",  # Replace with the Earthscope stream username
        "password": "<earthscope_password>"   # Replace with the Earthscope stream password
    },
    {
        "keywords": "pokemon",
        "match_all": True,
        "filter_semantics": ["name IN ['sturdy', 'damp', 'limber']", "IF name = 'sturdy' THEN alert = red"]
    }
]

## Initialize Clients


Here, we initialize the `APIClient` and `StreamingClient` that will handle the streams.


In [ ]:
# Initialize API Client
client = APIClient(base_url=API_URL, username=USERNAME, password=PASSWORD)

# Initialize Streaming Client
streaming = StreamingClient(pop_client=client)
print(f"User ID: {streaming.user_id}")

## Test Stream Creation, Consumption, and Deletion


In this section, we:
1. Create Kafka streams.
2. Consume messages from the topics.
3. Delete the streams after processing.


In [ ]:

async def test_create_and_consume_multiple_kafka_streams():
    for stream_config in streams_to_test:
        # Step 1: Create the Kafka stream
        result = await streaming.create_kafka_stream(
            keywords=stream_config.get("keywords", "").split(","),
            filter_semantics=stream_config.get("filter_semantics", []),
            match_all=stream_config.get("match_all", True),
            username=stream_config.get("username", None),
            password=stream_config.get("password", None)
        )

        if "error" in result:
            print(f"Error creating stream: {result['error']}")
            continue

        # Extract the topic and involved streams
        topic = result["topic"]
        involved_streams = result["involved_streams"]

        print(f"Stream created: {topic}")
        print(f"Involved streams: {involved_streams}")

        # Step 2: Consume messages from the Kafka topic
        print("\nConsuming messages from the Kafka topic...")
        consumer = streaming.consume_kafka_messages(topic)

        try:
            start_time = time.time()
            while True:
                if time.time() - start_time > 180:
                    print("Timeout reached while waiting for messages.")
                    break

                if not consumer.dataframe.empty:
                    print("Dataframe received:")
                    print(consumer.dataframe.head())
                    break

                await asyncio.sleep(1)
        finally:
            # Stop the consumer
            print("\nStopping the Kafka consumer...")
            consumer.stop()

        # Step 3: Delete the created Kafka stream
        print("\nDeleting the Kafka stream...")
        try:
            response = await streaming.delete_stream(topic)
            print(response["message"])
        except Exception as e:
            print(f"Error deleting stream {topic}: {e}")

        time.sleep(10)  # Wait some seconds to let the consumer clear its data

# Run the test
await test_create_and_consume_multiple_kafka_streams()